<a href="https://colab.research.google.com/github/masog1/PROGCOM-A/blob/main/ppt_9_gui_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# Juego: Piedra, Papel, Tijera (versión 9 elementos) con Tkinter
# Elementos: Piedra, Papel, Tijera, Pistola, Agua, Aire, Esponja, Humano, Fuego

import tkinter as tk
from tkinter import ttk, messagebox
from random import choice

# ----- Configuración del juego -----
ELEMENTOS = [
    "Piedra", "Papel", "Tijera", "Pistola", "Agua",
    "Aire", "Esponja", "Humano", "Fuego"
]

# Regla circular: cada elemento vence a los 4 siguientes en la lista (módulo 9)
VENCE_A = {}
n = len(ELEMENTOS)
for i, mov in enumerate(ELEMENTOS):
    VENCE_A[mov] = [ELEMENTOS[(i + j) % n] for j in range(1, 5)]  # 4 siguientes


class Juego9(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Piedra · Papel · Tijera — 9 elementos")
        self.resizable(False, False)

        # Estado
        self.puntos_usuario = 0
        self.puntos_cpu = 0
        self.objetivo = tk.IntVar(value=3)  # mejor de

        # ----- UI -----
        marco = ttk.Frame(self, padding=14)
        marco.grid()

        ttk.Label(
            marco, text="Versión extendida (9 elementos)",
            font=("Segoe UI", 16, "bold")
        ).grid(column=0, row=0, columnspan=3, pady=(0, 8))

        fila_opts = ttk.Frame(marco)
        fila_opts.grid(column=0, row=1, columnspan=3, sticky="w", pady=(0, 6))
        ttk.Label(fila_opts, text="Jugar a (mejor de):").grid(column=0, row=0, padx=(0, 6))
        ttk.Combobox(
            fila_opts, textvariable=self.objetivo, width=5, state="readonly",
            values=(1, 3, 5, 7)
        ).grid(column=1, row=0)

        self.lbl_marcador = ttk.Label(
            marco, text=self._texto_marcador(), font=("Segoe UI", 12, "bold")
        )
        self.lbl_marcador.grid(column=0, row=2, columnspan=3, pady=(0, 6))

        self.lbl_resultado = ttk.Label(
            marco, text="Elige tu jugada:", font=("Segoe UI", 11)
        )
        self.lbl_resultado.grid(column=0, row=3, columnspan=3, pady=(0, 8))

        # Botones 3x3 para los 9 elementos
        rejilla = ttk.Frame(marco)
        rejilla.grid(column=0, row=4, columnspan=3)
        for i, mov in enumerate(ELEMENTOS):
            ttk.Button(
                rejilla, text=mov, width=14, command=lambda m=mov: self.jugar(m)
            ).grid(row=i // 3, column=i % 3, padx=4, pady=4)

        # Historial
        ttk.Label(marco, text="Historial de rondas:").grid(column=0, row=5, columnspan=3, sticky="w", pady=(8, 0))
        self.txt_hist = tk.Text(marco, width=46, height=8, state="disabled")
        self.txt_hist.configure(font=("Consolas", 10))
        self.txt_hist.grid(column=0, row=6, columnspan=3, pady=(0, 8))

        # Controles inferiores
        controles = ttk.Frame(marco)
        controles.grid(column=0, row=7, columnspan=3)
        ttk.Button(controles, text="Reiniciar partida", command=self.reiniciar).grid(column=0, row=0, padx=4)
        ttk.Button(controles, text="Salir", command=self.destroy).grid(column=1, row=0, padx=4)

    # ----- Lógica -----
    def jugar(self, jugada_usuario: str):
        jugada_cpu = choice(ELEMENTOS)

        if jugada_cpu in VENCE_A[jugada_usuario]:
            self.puntos_usuario += 1
            msg = f"Ganaste: {jugada_usuario} vence a {jugada_cpu}"
        elif jugada_usuario in VENCE_A[jugada_cpu]:
            self.puntos_cpu += 1
            msg = f"Perdiste: {jugada_cpu} vence a {jugada_usuario}"
        else:
            msg = f"Empate: {jugada_usuario} = {jugada_cpu}"

        self.lbl_resultado.config(text=msg)
        self.lbl_marcador.config(text=self._texto_marcador())
        self._agregar_historial(jugada_usuario, jugada_cpu, msg)

        # ¿alguien alcanzó el objetivo?
        objetivo = int(self.objetivo.get())
        if self.puntos_usuario >= objetivo or self.puntos_cpu >= objetivo:
            if self.puntos_usuario > self.puntos_cpu:
                messagebox.showinfo("Partida terminada", f"¡Has ganado {self.puntos_usuario}-{self.puntos_cpu}!")
            else:
                messagebox.showinfo("Partida terminada", f"Has perdido {self.puntos_usuario}-{self.puntos_cpu}.")
            self.reiniciar(borrar_historial=False)

    def reiniciar(self, borrar_historial=True):
        self.puntos_usuario = 0
        self.puntos_cpu = 0
        self.lbl_marcador.config(text=self._texto_marcador())
        self.lbl_resultado.config(text="Elige tu jugada:")
        if borrar_historial:
            self.txt_hist.configure(state="normal")
            self.txt_hist.delete("1.0", tk.END)
            self.txt_hist.configure(state="disabled")

    def _agregar_historial(self, user: str, cpu: str, msg: str):
        self.txt_hist.configure(state="normal")
        self.txt_hist.insert(tk.END, f"Tú: {user:<8}  |  CPU: {cpu:<8}  →  {msg}\n")
        self.txt_hist.see(tk.END)
        self.txt_hist.configure(state="disabled")

    def _texto_marcador(self) -> str:
        return f"Marcador — Tú: {self.puntos_usuario} | CPU: {self.puntos_cpu}   (objetivo: {self.objetivo.get()})"


if __name__ == "__main__":
    Juego9().mainloop()


TclError: no display name and no $DISPLAY environment variable